In [1]:
from __future__ import division
from __future__ import print_function

import sys, os
sys.path.append('../')

import tensorflow as tf
import numpy as np
import gc

from models.neural_network import NeuralNetwork
from models.fcc import FCC
from models.util import normalize, avg_l2_distortion


os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#Seed used for all calculations of training and test point indices 
SEED = 14

Using TensorFlow backend.


ImportError: cannot import name avg_l2_distortion

In [ ]:
#Define params of model
input_shape = (545333)
num_classes = 2
eps = 10

In [ ]:
#Load model from disk (Note: Drebin takes a few minutes to load the dataset)
model_name = 'DREBIN'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = FCC(model_name=model_name, dataset='drebin')
print ('Total Model Params: %d' % model.num_params)
#model.load_model(model_save_path, weights_save_path) 
#epochs = 10
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)   

In [ ]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.test_data, model.test_labels)[1]))

In [ ]:
#Get training samples
num_train_samples = 100
data_indices = model.gen_rand_indices(low=0, high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)

In [ ]:
num_test_samples_per_class = 50
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)

#Get Regular, Noisy, FGSM, BIM, and CW test points
reg_data = model.test_data[test_indices]
noisy_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='NOISY', eps=eps)
#For JSMA we need to flip labels
jsma_tar = (model.test_labels[test_indices] == 0).astype(int)
jsma_data = model.generate_perturbed_data_binary(model.test_data[test_indices], jsma_tar,seed=SEED, perturbation='JSMA', eps=eps)
fgsm_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices],seed=SEED, perturbation='FGSM', eps=eps)
bim_a_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-A', iterations=10, eps=eps)
bim_b_data = model.generate_perturbed_data_binary(model.test_data[test_indices], model.test_labels[test_indices], seed=SEED, perturbation='BIM-B', iterations=10, eps=eps)

In [ ]:
#Reset tf.graph() as Cleverhans modifies the graph
tf.reset_default_graph()

#Reload the model and weights
model = FCC(model_name=model_name, dataset='drebin')
model.load_model(model_save_path, weights_save_path)    

In [ ]:
print (model.model.evaluate(reg_data,model.test_labels[test_indices]))
print (model.model.evaluate(noisy_data,model.test_labels[test_indices]))
print (model.model.evaluate(fgsm_data,model.test_labels[test_indices]))
print (model.model.evaluate(bim_a_data,model.test_labels[test_indices]))
print (model.model.evaluate(bim_b_data,model.test_labels[test_indices]))
print (model.model.evaluate(jsma_data,model.test_labels[test_indices]))



In [ ]:
#Get predictions
reg_preds = model.model.predict(reg_data.reshape(-1,*input_shape))
noisy_preds = model.model.predict(noisy_data.reshape(-1,*input_shape))
fgsm_preds = model.model.predict(fgsm_data.reshape(-1,*input_shape))
bim_a_preds = model.model.predict(bim_a_data.reshape(-1,*input_shape))
bim_b_preds = model.model.predict(bim_b_data.reshape(-1,*input_shape))
jsma_preds = model.model.predict(jsma_data.reshape(-1,*input_shape))


In [ ]:
#Convert preds to labels
reg_labels = np.zeros(reg_preds.shape)
reg_labels[np.arange(num_test_samples),np.argmax(reg_preds, axis=1)] = 1

noisy_labels = np.zeros(noisy_preds.shape)
noisy_labels[np.arange(num_test_samples),np.argmax(noisy_preds, axis=1)] = 1

fgsm_labels = np.zeros(fgsm_preds.shape)
fgsm_labels[np.arange(num_test_samples),np.argmax(fgsm_preds, axis=1)] = 1

bim_a_labels = np.zeros(bim_a_preds.shape)
bim_a_labels[np.arange(num_test_samples),np.argmax(bim_a_preds, axis=1)] = 1

bim_b_labels = np.zeros(bim_b_preds.shape)
bim_b_labels[np.arange(num_test_samples),np.argmax(bim_b_preds, axis=1)] = 1

jsma_labels = np.zeros(jsma_preds.shape)
jsma_labels[np.arange(num_test_samples),np.argmax(jsma_preds, axis=1)] = 1

In [ ]:
#Check preds to ensure adversarial samples were generated correctly
print (np.argmax(reg_preds, axis=1))
print (np.argmax(noisy_preds, axis=1))
print (np.argmax(fgsm_preds, axis=1))
print (np.argmax(bim_a_preds, axis=1))
print (np.argmax(bim_b_preds, axis=1))
print (np.argmax(jsma_preds, axis=1))

In [ ]:
#This is a large network, therefore we need to calculate cos_sim for one type and free memory
grads_train = model.get_gradients_wrt_params(train_data, train_data_labels)
#Hack to bypass memory issue, inplace modify
grads_train = normalize(grads_train)

#REG
grads_reg = model.get_gradients_wrt_params(reg_data, reg_labels)
grads_reg_nm = normalize(grads_reg)
grads_reg_norms = np.sqrt(np.dot(grads_reg, grads_reg.T)).diagonal()
cos_sim_reg = np.dot(grads_reg_nm, grads_train.T)
del grads_reg_nm, grads_reg
gc.collect()

#NOISY
grads_noisy = model.get_gradients_wrt_params(noisy_data, noisy_labels)
grads_noisy_nm = normalize(grads_noisy)
grads_noisy_norms = np.sqrt(np.dot(grads_noisy, grads_noisy.T)).diagonal()
cos_sim_noisy= np.dot(grads_noisy_nm, grads_train.T)
del grads_noisy_nm, grads_noisy
gc.collect()

#FGSM
grads_fgsm = model.get_gradients_wrt_params(fgsm_data, fgsm_labels)
grads_fgsm_nm = normalize(grads_fgsm)
grads_fgsm_norms = np.sqrt(np.dot(grads_fgsm, grads_fgsm.T)).diagonal()
cos_sim_fgsm = np.dot(grads_fgsm_nm, grads_train.T)
del grads_fgsm_nm, grads_fgsm
gc.collect()

#BIM-A
grads_bim_a = model.get_gradients_wrt_params(bim_a_data, bim_a_labels)
grads_bim_a_nm = normalize(grads_bim_a)
grads_bim_a_norms = np.sqrt(np.dot(grads_bim_a, grads_bim_a.T)).diagonal()
cos_sim_bim_a = np.dot(grads_bim_a_nm, grads_train.T)
del grads_bim_a_nm, grads_bim_a
gc.collect()

#BIM-B
grads_bim_b = model.get_gradients_wrt_params(bim_b_data, bim_b_labels)
grads_bim_b_nm = normalize(grads_bim_b)
grads_bim_b_norms = np.sqrt(np.dot(grads_bim_b, grads_bim_b.T)).diagonal()
cos_sim_bim_b = np.dot(grads_bim_b_nm, grads_train.T)
del grads_bim_b_nm, grads_bim_b
gc.collect()

#JSMA
grads_jsma = model.get_gradients_wrt_params(jsma_data, jsma_labels)
grads_jsma_nm = normalize(grads_jsma)
grads_jsma_norms = np.sqrt(np.dot(grads_jsma, grads_jsma.T)).diagonal()
cos_sim_jsma = np.dot(grads_jsma_nm, grads_train.T)
del grads_jsma_nm, grads_jsma
gc.collect()

In [ ]:
#Separate Using Cos Sim
eta = 0.59

count = 0.0
for i in range(num_test_samples):
    if np.max(cos_sim_reg[i]) > eta:
        count+=1.0
        
print ('Regular: %.4f' % ( count/num_test_samples))

count = 0.0
for i in range(num_test_samples):
    if np.max(cos_sim_noisy[i]) > eta:
        count+=1.0
    
print ('Noisy:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_fgsm[i]) > eta:
        count+=1.0
        
print ('FGSM:  %.4f' % ( count/num_test_samples))


count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_bim_a[i]) > eta:
        count+=1.0
        
print ('BIM-A:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_bim_b[i]) > eta:
        count+=1.0
        
print ('BIM-B:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_jsma[i]) > eta:
        count+=1.0
    
print ('JSMA: %.4f' % ( count/num_test_samples))

In [ ]:
#Separate using just norm
gamma = 1.0000024

count = 0.0
for i in range(num_test_samples):
    if grads_reg_norms[i] < gamma:
        count+=1.0
        
print ('Regular: %.4f' % ( count/num_test_samples))

count = 0.0
for i in range(num_test_samples):
    if grads_noisy_norms[i] < gamma:
        count+=1.0
    
print ('Noisy:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_fgsm_norms[i] < gamma:
        count+=1.0
        
print ('FGSM:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_bim_a_norms[i] < gamma:
        count+=1.0
        
print ('BIM-A:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_bim_b_norms[i] < gamma:
        count+=1.0
        
print ('BIM-B:  %.4f' % ( count/num_test_samples))


count=0.0
for i in range(num_test_samples):
    if grads_jsma_norms[i] < gamma:
        count+=1.0
    
print ('JSMA: %.4f' % ( count/num_test_samples))


In [ ]:
#Use both cos and norm

count = 0.0
for i in range(num_test_samples):
    if np.max(cos_sim_reg[i]) > eta and grads_reg_norms[i] < gamma:
        count+=1.0
        
print ('Regular: %.4f' % ( count/num_test_samples))

count = 0.0
for i in range(num_test_samples):
    if np.max(cos_sim_noisy[i]) > eta and grads_noisy_norms[i] < gamma:
        count+=1.0
    
print ('Noisy:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_fgsm[i]) > eta and grads_fgsm_norms[i] < gamma:
        count+=1.0
        
print ('FGSM:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_bim_a[i]) > eta and grads_bim_a_norms[i] < gamma:
        count+=1.0
        
print ('BIM-A:  %.4f' % ( count/num_test_samples))


count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_jsma[i]) > eta and grads_jsma_norms[i] < gamma:
        count+=1.0
    
print ('JSMA: %.4f' % ( count/num_test_samples))


In [ ]:
print ('Noisy: %.5f' % (avg_l2_distortion(reg_data, noisy_data)))
print ('FGSM: %.5f' % (avg_l2_distortion(reg_data, fgsm_data)))
print ('BIM-A: %.5f' % (avg_l2_distortion(reg_data, bim_a_data)))
print ('BIM-B: %.5f' % (avg_l2_distortion(reg_data, bim_b_data)))
print ('JSMA: %.5f' % (avg_l2_distortion(reg_data, jsma_data)))

In [ ]:
count = 0.0
for i in range(num_test_samples):
    if np.max(cos_sim_noisy[i]) > np.max(cos_sim_reg[i]):
        count+=1.0
    
print ('Noisy:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_fgsm[i]) > np.max(cos_sim_reg[i]):
        count+=1.0
        
print ('FGSM:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_bim_a[i])> np.max(cos_sim_reg[i]):
        count+=1.0
        
print ('BIM-A:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_bim_b[i]) > np.max(cos_sim_reg[i]):
        count+=1.0
        
print ('BIM-B:  %.4f' % ( count/num_test_samples))


count=0.0
for i in range(num_test_samples):
    if np.max(cos_sim_jsma[i]) > np.max(cos_sim_reg[i]):
        count+=1.0
    
print ('JSMA: %.4f' % ( count/num_test_samples))





In [ ]:
count = 0.0
for i in range(num_test_samples):
    if grads_noisy_norms[i] < grads_reg_norms[i]:
        count+=1.0
    
print ('Noisy:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_fgsm_norms[i] < grads_reg_norms[i]:
        count+=1.0
        
print ('FGSM:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_bim_a_norms[i] < grads_reg_norms[i]:
        count+=1.0
        
print ('BIM-A:  %.4f' % ( count/num_test_samples))

count=0.0
for i in range(num_test_samples):
    if grads_bim_b_norms[i] < grads_reg_norms[i]:
        count+=1.0
        
print ('BIM-B:  %.4f' % ( count/num_test_samples))


count=0.0
for i in range(num_test_samples):
    if grads_jsma_norms[i] < grads_reg_norms[i]:
        count+=1.0
    
print ('JSMA: %.4f' % ( count/num_test_samples))
